# **Cell 1: Mount**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Cell 2: train, test, generate**

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import pandas as pd
from torch.utils.data import DataLoader, random_split, Dataset
from PIL import Image
from google.colab import drive  # For Google Colab


# Define paths (adjust these as needed)
BASE_DIR = "/content/drive/My Drive/cse144-final"
TRAIN_DIR = os.path.join(BASE_DIR, "train/train")  # Training images in subfolders "0", "1", ..., "99"
TEST_DIR = os.path.join(BASE_DIR, "test/test")    # Test images (e.g., "0.jpg", "1.jpg", ...)
MODEL_PATH = os.path.join(BASE_DIR, "efficientnetb0_model.pth")
SUBMISSION_CSV = os.path.join(BASE_DIR, "submission.csv")

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define a simple transform for training and validation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom dataset to load images from a folder organized by class folders
class TrainDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.classes = sorted(os.listdir(root))
        self.samples = []
        for cls in self.classes:
            cls_path = os.path.join(root, cls)
            if os.path.isdir(cls_path):
                for fname in os.listdir(cls_path):
                    if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.webp')):
                        self.samples.append((os.path.join(cls_path, fname), int(cls)))
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# Custom dataset for test images (assumes all images are in a single folder)
class TestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.test_dir = test_dir
        self.image_paths = sorted(os.listdir(test_dir), key=lambda x: int(os.path.splitext(x)[0]))
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = os.path.join(self.test_dir, self.image_paths[idx])
        image = Image.open(path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.image_paths[idx]

# Load the training dataset
full_dataset = TrainDataset(TRAIN_DIR, transform=transform)
print("Total training images:", len(full_dataset))

# Split into training and validation sets (e.g., 80% train, 20% validation)
val_split = 0.2
num_total = len(full_dataset)
num_val = int(num_total * val_split)
num_train = num_total - num_val
train_dataset, val_dataset = random_split(full_dataset, [num_train, num_val])
print(f"Training images: {num_train}, Validation images: {num_val}")

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

# Load EfficientNet-B0 pretrained on ImageNet
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# Replace the classifier with a custom head for 100 classes
num_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.7),
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.7),
    nn.Linear(512, 100)
)
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Training loop
num_epochs = 50
best_val_loss = float('inf')
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    train_loss = running_loss / total
    train_acc = correct / total

    # Validation
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)
    val_loss = running_val_loss / total_val
    val_acc = correct_val / total_val

    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), MODEL_PATH)

# Load best model
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

# Prepare test dataset and DataLoader
test_dataset = TestDataset(TEST_DIR, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)

# Make predictions on test data and save submission CSV
predictions = []
with torch.no_grad():
    for images, img_names in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        for name, pred in zip(img_names, preds.cpu().numpy()):
            predictions.append((name, pred))

df = pd.DataFrame(predictions, columns=["ID", "Label"])
df.to_csv(SUBMISSION_CSV, index=False)

print("Predictions saved as", SUBMISSION_CSV)


Using device: cuda
Total training images: 1000
Training images: 800, Validation images: 200


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 56.2MB/s]


Epoch [1/50] | Train Loss: 4.6122 Acc: 0.0150 | Val Loss: 4.4805 Acc: 0.0800
Epoch [2/50] | Train Loss: 4.3155 Acc: 0.0862 | Val Loss: 4.1554 Acc: 0.0550
Epoch [3/50] | Train Loss: 3.8534 Acc: 0.1300 | Val Loss: 3.7863 Acc: 0.0450
Epoch [4/50] | Train Loss: 3.3988 Acc: 0.1663 | Val Loss: 3.4374 Acc: 0.1200
Epoch [5/50] | Train Loss: 3.0015 Acc: 0.2450 | Val Loss: 3.2017 Acc: 0.1650
Epoch [6/50] | Train Loss: 2.6614 Acc: 0.3475 | Val Loss: 3.0228 Acc: 0.1900
Epoch [7/50] | Train Loss: 2.3584 Acc: 0.4175 | Val Loss: 2.9693 Acc: 0.2350
Epoch [8/50] | Train Loss: 2.0611 Acc: 0.5200 | Val Loss: 2.8225 Acc: 0.2400
Epoch [9/50] | Train Loss: 1.8513 Acc: 0.5750 | Val Loss: 2.6391 Acc: 0.3100
Epoch [10/50] | Train Loss: 1.6274 Acc: 0.6575 | Val Loss: 2.6392 Acc: 0.2850
Epoch [11/50] | Train Loss: 1.4325 Acc: 0.7400 | Val Loss: 2.6214 Acc: 0.3150
Epoch [12/50] | Train Loss: 1.3430 Acc: 0.7575 | Val Loss: 2.7450 Acc: 0.3200
Epoch [13/50] | Train Loss: 1.1630 Acc: 0.8225 | Val Loss: 2.5640 Acc: 0.

<ipython-input-2-e4630a65b726>:154: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH))


Predictions saved as /content/drive/My Drive/cse144-final/submission.csv


# **Cell 3: save csv file**

In [ ]:
from google.colab import files

# Path to the CSV file
csv_path = "/content/drive/My Drive/cse144-final/submission.csv"

# Download the file
files.download(csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>